In [ ]:
import pickle, os
import numpy as np
import sys
import skimage as ski
from pathlib import Path
from PIL import Image
from matplotlib import pyplot as plt
import geopandas as gpd

# Linux  
#root_dir = f"/home/adriano/projeto_mestrado/modules"
#sys.path.insert(0, "/home/adriano/projeto_mestrado/modules")

# Windows  
root_dir = Path(r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")
sys.path.insert(0, r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")

from Background import background_generation as backgen
import functions

img_dir = f'{root_dir}/Images/retina/cutouts/training'
img_label = f'{root_dir}/Images/retina/cutouts/labels'

images_vector = functions.read_directories(img_dir)
label_vector = functions.read_directories(img_label)

print(mask_vector)

#for i in range(len(images_vector)):
for i in range(1):

    img = np.array(Image.open(f'{img_dir}/{images_vector[i]}'))
    mask = np.array(Image.open(f'{img_label}/{label_vector[i]}'))
    mask = np.array(Image.open('result.png'))
    
    cut = backgen.remove_artifacts2(img,mask)

    # Converting to Pillow Image
    cutout = Image.fromarray(cut)

    plt.figure(figsize=[10, 10])
    plt.imshow(cut, 'gray',vmin=0, vmax=255)
    plt.axis('off')
    cutout.save('teste.png')
    plt.show()

    


In [53]:
from PIL import Image, ImageDraw
import numpy as np
import pickle, os
import numpy as np
import sys
import skimage as ski
from pathlib import Path
from PIL import Image
from matplotlib import pyplot as plt

import functions


# Linux  
#root_dir = f"/home/adriano/projeto_mestrado/modules"
#sys.path.insert(0, "/home/adriano/projeto_mestrado/modules")

# Windows  
root_dir = Path(r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")
sys.path.insert(0, r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")


def recorte_circular(imagem_caminho, posicao):

    img_out = f'{root_dir}/Images/retina/cutouts/backgrounds/teste'
    # Abre a imagem
    imagem = Image.open(imagem_caminho).convert("RGBA")
    
    # Cria a máscara em branco
    mascara = Image.new("L", imagem.size, 0)
    
    # Define o tamanho e o centro do círculo
    largura, altura = imagem.size[0], imagem.size[1]
    raio = (min(largura, altura) // 2 )-70
    centro = (largura // 2, altura // 2)
    
    # Desenha o círculo na máscara
    draw = ImageDraw.Draw(mascara)
    draw.ellipse((centro[0] - raio, centro[1] - raio, centro[0] + raio, centro[1] + raio), fill=255)
    
    # Aplica a máscara à imagem
    imagem_arr = np.array(imagem)
    mascara_arr = np.array(mascara)
    imagem_arr[:, :, 3] = mascara_arr    
    
    # Cria a imagem resultante
    imagem_resultante = Image.fromarray(imagem_arr)

    imagem_rgb = Image.new("RGB", imagem_resultante.size, (255, 255, 255))
    imagem_rgb.paste(imagem_resultante, mask=imagem_resultante.split()[3])  # 3 is the alpha channel

    
    #imagem_resultante = Image.fromarray(imagem_arr)
    
    # Salva a imagem resultante
    imagem_rgb.save(f'{img_out}/{posicao+1}.tiff')


img_dir = f'{root_dir}/Images/retina/backgrounds'


images_vector = functions.read_directories(img_dir)

for i in range(len(images_vector)):
# Exemplo de uso
    recorte_circular(f'{img_dir}/{images_vector[i]}', i)


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
import numpy as np
import pickle, os
import numpy as np
import sys
import skimage as ski
from pathlib import Path
from PIL import Image
from matplotlib import pyplot as plt

import functions


# Linux  
#root_dir = f"/home/adriano/projeto_mestrado/modules"
#sys.path.insert(0, "/home/adriano/projeto_mestrado/modules")

# Windows  
root_dir = Path(r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")
sys.path.insert(0, r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")

img_dir = f'{root_dir}/Images/retina/backgrounds'

def remove_artifacts(original, binary):

    original = cv2.imread(imagem_original)

    binary = cv2.imread(imagem_binaria, 0)  # Carrega como escala de cinza

    mask_pil = Image.open(binary)
    mask = np.array(mask_pil)

    
     # Verificando as dimensões das imagens
    if original.shape[:2] != binary.shape:
        raise ValueError("As dimensões das imagens não são compatíveis.")
    
    original[mask > 0] = 0

    # Percorrendo os pixels e removendo os artefatos
    #for i in range(original.shape[0]):
        #for j in range(original.shape[1]):
           
            #if binary[i, j] == 255:  # Verifica se o pixel é branco na imagem binária
                # Substituindo o pixel da imagem original pela média dos pixels vizinhos
                #print(binary[i, j])
                #original[i, j] = np.mean(original[max(0, i-1):min(original.shape[0], i+2),
                                                  # max(0, j-1):min(original.shape[1], j+2)])

    return original

# Exemplo de uso
imagem_original = f"{root_dir}/Images/retina/cutouts/training/21.tiff"
imagem_binaria = f"{root_dir}/Images/retina/labels/21_manual1.tiff"



imagem_corrigida = remove_artifacts(imagem_original, imagem_binaria)

# Salvando a nova imagem corrigida
cv2.imwrite("corrigida.tiff", imagem_corrigida)


In [90]:
import cv2
import numpy as np
from PIL import Image

# Caminhos dos arquivos
imagem_original = f"{root_dir}/Images/retina/cutouts/training/21.tiff"
imagem_binaria = f"{root_dir}/Images/retina/labels/21_manual1.tiff"



original = cv2.imread(imagem_original)

#binary = cv2.imread(imagem_binaria, 0)  # Carrega como escala de cinza

# Carregar a imagem binária (máscara) usando PIL
mask_pil = Image.open(imagem_binaria).convert('L')  # Converter para escala de cinza
mask = np.array(mask_pil)

# Verificar se as dimensões da imagem e da máscara são iguais
if original.shape[:2] != mask.shape[:2]:
    raise ValueError("A originalm original e a máscara têm dimensões diferentes")

# Expandir a máscara para ter a mesma forma que a originalm original
mask = np.expand_dims(mask, axis=2)
mask = np.repeat(mask, 3, axis=2)

# Aplicar a máscara para remover os vasos (definindo os pixels da máscara como 0 na originalm original)
original[mask > 0] = 0

# Salvar a originalm resultante
output_path = 'originalm_sem_vasos.jpg'
cv2.imwrite(output_path, original)

print(f"Imagem processada e salva em {output_path}")

Imagem processada e salva em originalm_sem_vasos.jpg


In [89]:
import cv2
import numpy as np
from PIL import Image

# Caminhos dos arquivos
imagem_original = f"{root_dir}/Images/retina/cutouts/training/21.tiff"
imagem_binaria = f"{root_dir}/Images/retina/labels/21_manual1.tiff"



original = cv2.imread(imagem_original)

#binary = cv2.imread(imagem_binaria, 0)  # Carrega como escala de cinza

# Expandir a máscara para ter a mesma forma que a imagem original
mask = np.expand_dims(mask, axis=2)
mask = np.repeat(mask, 3, axis=2)

# Criar uma cópia da imagem original para a saída
output_image = original.copy()

# Encontra as coordenadas dos pixels que precisam ser preenchidos
indices = np.where(mask == 255)

# Iterar sobre todos os pixels que precisam ser preenchidos
for y, x in zip(indices[0], indices[1]):
    # Extrair uma pequena vizinhança de 3x3 em torno do pixel branco
    neighborhood = original[max(y-1, 0):min(y+2, original.shape[0]), max(x-1, 0):min(x+2, original.shape[1])]
    
    # Calcular a média dos pixels na vizinhança, excluindo os brancos
    if neighborhood.size > 0:
        mean_value = np.mean(neighborhood[neighborhood != 0])
        if not np.isnan(mean_value):
            output_image[y, x] = mean_value+50

# Salvar a imagem resultante
output_path = 'imagem_sem_vasosteste.jpg'
cv2.imwrite(output_path, output_image)

print(f"Imagem processada e salva em {output_path}")

Imagem processada e salva em imagem_sem_vasosteste.jpg
